3. Who is performing co-visits? Are they generally performed by the same provider or discipline or by multiple providers or disciplines? Is it more likely for a co-visit to occur if vendor services are within the same agency?

In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [70]:
# Importing csv files from TEIS

df = pd.read_csv('../data/TEIS_IFSP_Teaming_Jan_2022-_Jan_2023.xlsx - Compiled_IFSP Teaming.csv')

In [71]:
# View columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9543 entries, 0 to 9542
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CHILD_ID              9543 non-null   int64 
 1   SERVICE_DATE          9543 non-null   object
 2   START_TIME            9532 non-null   object
 3   END_TIME              9532 non-null   object
 4   SERVICE_SETTING       9533 non-null   object
 5   SERVICE DELIVERY      9543 non-null   object
 6   ADD_USER              9543 non-null   object
 7   ADD_DATE              9543 non-null   object
 8   IFSP Teaming ID name  9543 non-null   object
 9   Agency Name           9543 non-null   object
 10  Agency Type           9543 non-null   object
 11  Service Name          9543 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.8+ KB


In [72]:
# Determine if providers have more visitis or co-visits

providers = df.groupby(['Agency Name', 'IFSP Teaming ID name']).size()
providers

Agency Name                                   IFSP Teaming ID name
BEST (Blind-Visually Impaired Early Svcs Tn)  Co-Visit                 44
                                              IEP Meeting               4
                                              IFSP Meeting             28
BRIDGES - Tennessee Tech University           Co-Visit                 12
                                              IEP Meeting               6
                                                                     ... 
WAVES, Inc.                                   Co-Visit                169
                                              IEP Meeting              40
                                              IFSP Meeting            220
WeeCommunicate, LLC                           Co-Visit                  1
Words for Life Speech & Language Center, LLC  Co-Visit                  1
Length: 156, dtype: int64

In [73]:
# Creating an identifying number for visits

covisits = df[df['IFSP Teaming ID name'] == 'Co-Visit']

covisits.head()

,CHILD_ID,SERVICE_DATE,START_TIME,END_TIME,SERVICE_SETTING,SERVICE DELIVERY,ADD_USER,ADD_DATE,IFSP Teaming ID name,Agency Name,Agency Type,Service Name
1,457136,2/15/2022,14:00:00,15:00:00,Community-based,Service Delivered,LEECAROL1,2/16/22 5:29,Co-Visit,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community
6,458850,2/15/2022,14:00:00,14:30:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:07,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community
7,458851,2/15/2022,14:15:00,14:45:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:08,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community
8,459167,1/26/2022,9:30:00,10:00:00,Telehealth,Service Delivered,HINESKONSTANCE,1/26/22 11:08,Co-Visit,LeBonheur EI & Dev (LEAD) (EIRA),EIRA,Physical Therapy Assistant
9,459167,1/26/2022,9:00:00,10:00:00,Telehealth,Service Delivered,GLASSKATHERINE,2/2/22 11:57,Co-Visit,LeBonheur EI & Dev (LEAD) (vendor),Independent,Speech Therapy


In [74]:
# Sorting the visits by CHILD_ID and SERVICE_DATE

covisits = covisits.sort_values(['CHILD_ID', 'SERVICE_DATE'])

covisits.head()

,CHILD_ID,SERVICE_DATE,START_TIME,END_TIME,SERVICE_SETTING,SERVICE DELIVERY,ADD_USER,ADD_DATE,IFSP Teaming ID name,Agency Name,Agency Type,Service Name
1,457136,2/15/2022,14:00:00,15:00:00,Community-based,Service Delivered,LEECAROL1,2/16/22 5:29,Co-Visit,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community
6,458850,2/15/2022,14:00:00,14:30:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:07,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community
7,458851,2/15/2022,14:15:00,14:45:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:08,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community
8,459167,1/26/2022,9:30:00,10:00:00,Telehealth,Service Delivered,HINESKONSTANCE,1/26/22 11:08,Co-Visit,LeBonheur EI & Dev (LEAD) (EIRA),EIRA,Physical Therapy Assistant
9,459167,1/26/2022,9:00:00,10:00:00,Telehealth,Service Delivered,GLASSKATHERINE,2/2/22 11:57,Co-Visit,LeBonheur EI & Dev (LEAD) (vendor),Independent,Speech Therapy


In [75]:
# Creating identying number for covisits


covisits = covisits.reset_index(drop=True)

visit_id = 1
visit_ids = []

for index, row in covisits.iterrows():
    if index == 0:
        visit_ids.append(visit_id)
    elif row['CHILD_ID'] == covisits.iloc[index - 1]['CHILD_ID'] and row['SERVICE_DATE'] == covisits.iloc[index - 1]['SERVICE_DATE']:
        visit_ids.append(visit_id)
    else:
        visit_id += 1
        visit_ids.append(visit_id)

covisits['VISIT_ID'] = visit_ids

In [76]:
covisits

,CHILD_ID,SERVICE_DATE,START_TIME,END_TIME,SERVICE_SETTING,SERVICE DELIVERY,ADD_USER,ADD_DATE,IFSP Teaming ID name,Agency Name,Agency Type,Service Name,VISIT_ID
0,457136,2/15/2022,14:00:00,15:00:00,Community-based,Service Delivered,LEECAROL1,2/16/22 5:29,Co-Visit,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community,1
1,458850,2/15/2022,14:00:00,14:30:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:07,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community,2
2,458851,2/15/2022,14:15:00,14:45:00,Telehealth,Service Delivered,BUCKCYNTHIA,2/15/22 18:08,Co-Visit,"Physical Therapy Services, PA (EIRA)",EIRA,Dev. Therapy Prof - Home/Community,3
3,459167,1/26/2022,9:30:00,10:00:00,Telehealth,Service Delivered,HINESKONSTANCE,1/26/22 11:08,Co-Visit,LeBonheur EI & Dev (LEAD) (EIRA),EIRA,Physical Therapy Assistant,4
4,459167,1/26/2022,9:00:00,10:00:00,Telehealth,Service Delivered,GLASSKATHERINE,2/2/22 11:57,Co-Visit,LeBonheur EI & Dev (LEAD) (vendor),Independent,Speech Therapy,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,540954,1/27/2023,10:35:00,11:33:00,Other,Service Delivered,SPRADLINGSARAH,1/27/23 15:21,Co-Visit,Rainbow Early Intervention (HATS),EIRA,Dev. Therapy Prof - Home/Community,3552
4087,541337,1/10/2023,15:00:00,16:00:00,Home,Service Delivered,WESTAMY,1/12/23 17:55,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,3553
4088,541511,1/19/2023,13:30:00,14:30:00,Home,Service Delivered,WESTAMY,1/20/23 16:11,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,3554
4089,541890,1/19/2023,9:00:00,10:00:00,Home,Service Delivered,WESTAMY,1/20/23 10:23,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,3555


In [77]:
# Create a count of how many types of visits each CHILD ID has

ccount = df.groupby(['CHILD_ID', 'IFSP Teaming ID name']).size()

ccount



CHILD_ID  IFSP Teaming ID name
457136    Co-Visit                1
          IEP Meeting             1
457585    IEP Meeting             1
457852    IEP Meeting             1
458051    IFSP Meeting            1
                                 ..
541337    Co-Visit                1
541511    Co-Visit                1
541890    Co-Visit                1
542077    IFSP Meeting            1
542122    Co-Visit                1
Length: 5843, dtype: int64

In [78]:
# Create a count of how many types of visits and agencies each CHILD ID has

cpcount = df.groupby(['CHILD_ID', 'IFSP Teaming ID name', 'Agency Name']).size()

cpcount.head(20)

CHILD_ID  IFSP Teaming ID name  Agency Name                                      
457136    Co-Visit              Siskin Children's Institute (EIRA)                   1
          IEP Meeting           Siskin Children's Institute (EIRA)                   1
457585    IEP Meeting           Siskin Children's Institute (EIRA)                   1
457852    IEP Meeting           LTVEC                                                1
458051    IFSP Meeting          First Steps, Inc (EIRA)                              1
458230    IFSP Meeting          Sunshine Early Intervention - Knox Cty ARC           1
458850    Co-Visit              Physical Therapy Services, PA (EIRA)                 1
458851    Co-Visit              Physical Therapy Services, PA (EIRA)                 1
459167    Co-Visit              LeBonheur EI & Dev (LEAD) (EIRA)                     1
                                LeBonheur EI & Dev (LEAD) (vendor)                   4
                                Shelby Residenti

In [79]:
# Create a count of how many types of visits and agencies and services each CHILD ID has

cpscount = df.groupby(['CHILD_ID', 'IFSP Teaming ID name', 'Agency Name', 'Service Name']).size()

cpscount.head(20)

CHILD_ID  IFSP Teaming ID name  Agency Name                                        Service Name                      
457136    Co-Visit              Siskin Children's Institute (EIRA)                 Dev. Therapy Prof - Home/Community    1
          IEP Meeting           Siskin Children's Institute (EIRA)                 Dev. Therapy Prof - Home/Community    1
457585    IEP Meeting           Siskin Children's Institute (EIRA)                 Dev. Therapy Prof - Home/Community    1
457852    IEP Meeting           LTVEC                                              Dev. Therapy Prof - Home/Community    1
458051    IFSP Meeting          First Steps, Inc (EIRA)                            Dev. Therapy Prof - Home/Community    1
458230    IFSP Meeting          Sunshine Early Intervention - Knox Cty ARC         Dev. Therapy Prof - Home/Community    1
458850    Co-Visit              Physical Therapy Services, PA (EIRA)               Dev. Therapy Prof - Home/Community    1
458851    Co-Visit   

In [80]:
# Create a count of ISFP Meeting Types

ifsp = df.groupby('IFSP Teaming ID name').size()
ifsp

IFSP Teaming ID name
Co-Visit        4091
IEP Meeting      632
IFSP Meeting    4820
dtype: int64

In [81]:
# Create a running total to plot number of total visits for child by day to see how many are multiples

df2 = df.reset_index(drop=True)

df2 = df2.sort_values(['CHILD_ID', 'SERVICE_DATE'])

visit_id = 1
visit_ids = []

for index, row in df2.iterrows():
    if index == 0:
        visit_ids.append(visit_id)
    elif row['CHILD_ID'] == df2.iloc[index - 1]['CHILD_ID'] and row['SERVICE_DATE'] == df2.iloc[index - 1]['SERVICE_DATE']:
        visit_ids.append(visit_id)
    else:
        visit_id += 1
        visit_ids.append(visit_id)

df2['VISIT_ID'] = visit_ids

In [82]:
df2

,CHILD_ID,SERVICE_DATE,START_TIME,END_TIME,SERVICE_SETTING,SERVICE DELIVERY,ADD_USER,ADD_DATE,IFSP Teaming ID name,Agency Name,Agency Type,Service Name,VISIT_ID
0,457136,1/21/2022,9:30:00,10:30:00,Community-based,Service Delivered,LEECAROL1,1/21/22 17:54,IEP Meeting,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community,1
1,457136,2/15/2022,14:00:00,15:00:00,Community-based,Service Delivered,LEECAROL1,2/16/22 5:29,Co-Visit,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community,2
2,457585,1/24/2022,8:30:00,9:30:00,Telehealth,Service Delivered,MILLARDCATHERINE,1/24/22 14:18,IEP Meeting,Siskin Children's Institute (EIRA),EIRA,Dev. Therapy Prof - Home/Community,3
3,457852,1/12/2022,14:00:00,15:00:00,Community-based,Service Delivered,HEMELKATIE1,1/18/22 10:39,IEP Meeting,LTVEC,EIRA,Dev. Therapy Prof - Home/Community,4
4,458051,1/12/2022,12:00:00,13:00:00,Telehealth,Service Delivered,JONESMEGAN,1/12/22 13:46,IFSP Meeting,"First Steps, Inc (EIRA)",EIRA,Dev. Therapy Prof - Home/Community,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9538,541337,1/10/2023,15:00:00,16:00:00,Home,Service Delivered,WESTAMY,1/12/23 17:55,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,8493
9539,541511,1/19/2023,13:30:00,14:30:00,Home,Service Delivered,WESTAMY,1/20/23 16:11,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,8494
9540,541890,1/19/2023,9:00:00,10:00:00,Home,Service Delivered,WESTAMY,1/20/23 10:23,Co-Visit,"Kid's Putnam, Inc.",EIRA,Dev. Therapy Prof - Home/Community,8495
9541,542077,1/30/2023,14:30:00,15:20:00,Telehealth,Service Delivered,FANELLASAMANTHA,1/30/23 15:21,IFSP Meeting,Pediatric Language Clinic,EIRA,Dev. Therapy Prof - Home/Community,8496


In [84]:
# Create a count of how many covists for agencies each CHILD ID has

cpcvcount = covisits.groupby(['CHILD_ID', 'Agency Name']).size()

cpcvcount.head(20)

CHILD_ID  Agency Name                                      
457136    Siskin Children's Institute (EIRA)                   1
458850    Physical Therapy Services, PA (EIRA)                 1
458851    Physical Therapy Services, PA (EIRA)                 1
459167    LeBonheur EI & Dev (LEAD) (EIRA)                     1
          LeBonheur EI & Dev (LEAD) (vendor)                   4
          Shelby Residential & Voc Svcs(SRVS) (EIRA)           1
459566    Signal Centers, Inc.                                 1
460456    Susan Gray School - Vanderbilt Univ                  1
460584    LeBonheur EI & Dev (LEAD) (vendor)                   1
          Support Solutions of the Mid-South (CS Patterson)    1
460965    LeBonheur EI & Dev (LEAD) (vendor)                   5
461124    Kid's Putnam, Inc.                                   2
462527    BRIDGES - Tennessee Tech University                  1
462846    Vanderbilt Bill Wilkerson Center                     1
462915    BEST (Blind-Visually

In [88]:
# Create a count of how many covists for agencies and services each CHILD ID has

cpscvcount = covisits.groupby(['CHILD_ID', 'Agency Name', 'Service Name']).size()

cpscvcount.head(30)

CHILD_ID  Agency Name                                        Service Name                      
457136    Siskin Children's Institute (EIRA)                 Dev. Therapy Prof - Home/Community    1
458850    Physical Therapy Services, PA (EIRA)               Dev. Therapy Prof - Home/Community    1
458851    Physical Therapy Services, PA (EIRA)               Dev. Therapy Prof - Home/Community    1
459167    LeBonheur EI & Dev (LEAD) (EIRA)                   Physical Therapy Assistant            1
          LeBonheur EI & Dev (LEAD) (vendor)                 Occupational Therapy                  2
                                                             Speech Therapy                        2
          Shelby Residential & Voc Svcs(SRVS) (EIRA)         Dev. Therapy Prof - Home/Community    1
459566    Signal Centers, Inc.                               Dev. Therapy Prof - Home/Community    1
460456    Susan Gray School - Vanderbilt Univ                Dev. Therapy Prof - Home/Community 

In [91]:
#Create a dataframe with the Child ID, Agency completing service, Service Provided, and the count of visits.

cas_count = pd.DataFrame(cpscvcount)

cas_count = cas_count.reset_index().rename(columns={'CHILD_ID': 'Child ID', 0: 'Count', 'Agency Name': 'Agency', 'Service Name': 'Service'})

cas_count

,Child ID,Agency,Service,Count
0,457136,Siskin Children's Institute (EIRA),Dev. Therapy Prof - Home/Community,1
1,458850,"Physical Therapy Services, PA (EIRA)",Dev. Therapy Prof - Home/Community,1
2,458851,"Physical Therapy Services, PA (EIRA)",Dev. Therapy Prof - Home/Community,1
3,459167,LeBonheur EI & Dev (LEAD) (EIRA),Physical Therapy Assistant,1
4,459167,LeBonheur EI & Dev (LEAD) (vendor),Occupational Therapy,2
...,...,...,...,...
2027,540954,Rainbow Early Intervention (HATS),Dev. Therapy Prof - Home/Community,1
2028,541337,"Kid's Putnam, Inc.",Dev. Therapy Prof - Home/Community,1
2029,541511,"Kid's Putnam, Inc.",Dev. Therapy Prof - Home/Community,1
2030,541890,"Kid's Putnam, Inc.",Dev. Therapy Prof - Home/Community,1
